# Train DL

> A collection of deep learning tools via Fastai

In [ ]:
#| default_exp dl

In [ ]:
#| hide
import sys
sys.path.append("/notebooks/katlas")
from nbdev.showdoc import *
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
#| output: False
from fastbook import *
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from scipy.stats import spearmanr,pearsonr
from katlas.core import *
from katlas.feature import *
from sklearn.model_selection import *
import torch.nn.init as init
from katlas.train import *

In [ ]:
#| export
def seed_everything(seed=123):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
seed_everything()

## Dataset

In [ ]:
#| export
class GeneralDataset:
    def __init__(self, 
                 df, # a dataframe of values
                 feat_col, # feature columns
                 target_col=None # Will return test set for prediction if target col is None
                ):
        "A general dataset that can be applied to any dataframe"
        
        self.test = False if target_col is not None else True
        
        self.X = df[feat_col].values 
        self.y = df[target_col].values if not self.test else None
        
        self.len = df.shape[0]

    def __len__(self):
        return self.len

    def __getitem__(self, index):
        X = torch.Tensor(self.X[index])
        if self.test:
            return X
        else:
            y = torch.Tensor(self.y[index])
            return X, y

In [ ]:
# df = pd.read_parquet('train/scaled_t5.parquet')

# feat_col = df.columns[199:]
# target_col = df.columns[1:199]

# info = Data.get_kinase_info_full()
# info = info.query('in_paper ==1')
# info = df[['kinase']].merge(info,'left')

In [ ]:
# #| output: False
# splits = get_splits(info, stratified = 'category')
# split = splits[0]

In [ ]:
# train = df.loc[split[0]]
# valid = df.loc[split[1]]

# train.shape, valid.shape

# train_ds = GeneralDataset(train, feat_col, target_col)
# valid_ds = GeneralDataset(valid, feat_col, target_col)

# len(train_ds),len(valid_ds)

# dls = DataLoaders.from_dsets(train_ds, valid_ds, num_workers=4,bs=32)

# for batch in dls.train: # or dls.valid
#     break

# batch[0].shape, batch[1].shape

## Models

### Multi layer perceptron

In [ ]:
#| export
def MLP_1(num_features, 
          num_targets,
          hidden_units = [512, 218],
          dp = 0.2):
    
    # Start with the first layer from num_features to the first hidden layer
    layers = [
        nn.Linear(num_features, hidden_units[0]),
        nn.BatchNorm1d(hidden_units[0]),
        nn.Dropout(dp),
        nn.PReLU()
    ]
    
    # Loop over hidden units to create intermediate layers
    for i in range(len(hidden_units) - 1):
        layers.extend([
            nn.Linear(hidden_units[i], hidden_units[i+1]),
            nn.BatchNorm1d(hidden_units[i+1]),
            nn.Dropout(dp),
            nn.PReLU()
        ])
    
    # Add the output layer
    layers.append(nn.Linear(hidden_units[-1], num_targets))
    
    model = nn.Sequential(*layers)
    
    return model

In [ ]:
# hidden_units = [512]

In [ ]:
# num_feat = len(feat_col)
# num_target = len(target_col)

In [ ]:
# model = MLP_1(num_feat, num_target, hidden_units)

In [ ]:
# To train
# learn = Learner(dls.cuda(), model.cuda(), mse)

# learn.fit_one_cycle(5)

### CNN1D

In [ ]:
#| export
class CNN1D_1(Module):
    
    def __init__(self, 
                 num_features, # this does not matter, just for format
                 num_targets):

        self.conv1 = nn.Conv1d(in_channels=1, out_channels=3, kernel_size=3, dilation=1, padding=1, stride=1)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv1d(in_channels=3, out_channels=8, kernel_size=3, dilation=1, padding=1, stride=1)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.flatten = Flatten()
        self.fc1 = nn.Linear(in_features = int(8 * num_features/4), out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=num_targets)

    def forward(self, x):
        x = x.unsqueeze(1) # need shape (bs, 1, num_features) for CNN
        x = self.pool1(nn.functional.relu(self.conv1(x)))
        x = self.pool2(nn.functional.relu(self.conv2(x)))
        # x = torch.flatten(x, 1)
        x = self.flatten(x)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
# model = CNN1D_1(num_feat, num_target)

In [ ]:
#| export
#| hide
def ConvBlock(in_channels, 
              out_channels, 
              kernel_size, 
              stride, 
              padding, 
              dropout_rate):
    return nn.Sequential(
        nn.BatchNorm1d(in_channels),
        nn.Dropout(dropout_rate),
        nn.utils.weight_norm(nn.Conv1d(in_channels, out_channels, kernel_size, stride, padding)),
        nn.ReLU()
    )

In [ ]:
#| export
class CNN1D_2(Module):
    
    def __init__(self, 
                 num_features, 
                 num_targets, 
                 hidden_size=4096):

        cha_1 = 256
        cha_2 = 512
        cha_3 = 512

        cha_1_reshape = int(hidden_size/cha_1)
        cha_po_1 = int(hidden_size/cha_1/2)
        cha_po_2 = int(hidden_size/cha_1/2/2) * cha_3

        self.cha_1 = cha_1
        self.cha_2 = cha_2
        self.cha_3 = cha_3
        self.cha_1_reshape = cha_1_reshape
        self.cha_po_1 = cha_po_1
        self.cha_po_2 = cha_po_2

        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dropout1 = nn.Dropout(0.1)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))

        self.convBlock1 = ConvBlock(cha_1, cha_2, kernel_size=5, stride=1, padding=2, dropout_rate=0.1)

        self.ave_po_c1 = nn.AdaptiveAvgPool1d(output_size = cha_po_1)

        self.convBlock2 = ConvBlock(cha_2, cha_2, kernel_size=3, stride=1, padding=1, dropout_rate=0.1)
        self.convBlock3 = ConvBlock(cha_2, cha_2, kernel_size=3, stride=1, padding=1, dropout_rate=0.3)
        self.convBlock4 = ConvBlock(cha_2, cha_3, kernel_size=5, stride=1, padding=2, dropout_rate=0.2)

        self.max_po_c2 = nn.MaxPool1d(kernel_size=4, stride=2, padding=1)

        self.flt = nn.Flatten()

        self.batch_norm3 = nn.BatchNorm1d(cha_po_2)
        self.dropout3 = nn.Dropout(0.2)
        self.dense3 = nn.utils.weight_norm(nn.Linear(cha_po_2, num_targets))

        # Manual weights initialization
        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm1d): # initialize for clarification
                init.constant_(m.weight, 1)
                init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.celu(self.dense1(x), alpha=0.06)

        x = x.reshape(x.shape[0],self.cha_1, self.cha_1_reshape)

        x = self.convBlock1(x)

        x = self.ave_po_c1(x)

        x = self.convBlock2(x)
        x_s = x  # Saving for later use

        x = self.convBlock3(x)

        x = self.convBlock4(x)
        x = x * x_s  # Using saved value

        # Max pooling
        x = self.max_po_c2(x)

        # Flatten before final block
        x = self.flt(x)

        # Final block
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)

        return x

In [ ]:
# model = CNN1D_2(num_feat, num_target)

## DL Trainer

In [ ]:
#| export
def train_dl(df, 
            feat_col, 
            target_col,
            split, 
            model, 
             bs = 32,
            loss = mse, 
            save = None, # models/{save}.pth
             params = {'n_epoch': 4} # params in fastai Learner, e.g., lr_max
              ):
    
    train = df.loc[split[0]]
    valid = df.loc[split[1]]
    
    train_ds = GeneralDataset(train, feat_col, target_col)
    valid_ds = GeneralDataset(valid, feat_col, target_col)

    dls = DataLoaders.from_dsets(train_ds, valid_ds, bs=bs, num_workers=4)
    
    learn = Learner(dls.cuda(), model.cuda(), loss, metrics= [PearsonCorrCoef(),SpearmanCorrCoef()] )
    
    lr_max = learn.lr_find()
    plt.show()
    plt.close()
    print(lr_max)
    
    if 'lr_max' not in params:
        params['lr_max'] = lr_max
    
    print(f"lr_find is :{lr_max}, params lr is {params['lr_max']}")
    learn.fit_one_cycle(**params)
    
    if save is not None:
        learn.save(save)
    
    return learn

In [ ]:
# model = CNN1D_2(num_feat, num_target)

In [ ]:
# params = {'n_epoch': 14, 
#           'lr_max': 2e-2,
#           'cbs': [SaveModelCallback(fname = 'best')] # save best model
#          }

In [ ]:
# learn = train_dl(df,feat_col,target_col, split, model, save = 'final', params = params)

## DL Predict

In [ ]:
#| export
def predict_dl(df, 
               feat_col, 
               model, # model architecture
               model_pth, # only name, not with .pth
              ):
    test_dset = GeneralDataset(df,feat_col)
    test_dl = torch.utils.data.DataLoader(test_dset)
    
    
    learn = Learner(None, model.cuda(), loss_func=1)
    learn.load(model_pth)
    
    learn.model.eval()
    
    preds = []
    for data in test_dl:
        inputs = data.cuda()
        outputs = learn.model(inputs) #learn.model(x).sigmoid().detach().cpu().numpy()

        preds.append(outputs.detach().cpu().numpy())

    preds = np.concatenate(preds)
    preds = pd.DataFrame(preds)

    return preds

In [ ]:
# pred = predict_dl(df,feat_col,model,'best')

# target = df[target_col]

# target = pd.DataFrame(target.values)

# score_all(target,pred)

# score_each(target,pred).head()

## DL CV

In [ ]:
#| export
def train_dl_cv(df, 
                feat_col, 
                target_col, 
                splits, 
                model_fn, # functions like lambda x: return MLP_1(num_feat, num_target)
                save = 'model', # model name
                bs=32, 
                params={}):
    
    y = df[target_col]
    OOF = np.zeros(y.shape)  # Initialize an array with zeros, which will store the OOF predictions.
    
    for fold,split in enumerate(splits):

        print(f'------fold{fold}------')
        
        model = model_fn()
        fname = f'{save}_fold{fold}_best'
        
        params['cbs'] = [SaveModelCallback(fname=fname)] 

        learn = train_dl(df,feat_col,target_col, split, model ,bs = bs, params = params)

        # predict on valid split
        valid_df = df.loc[split[1]]
        pred = predict_dl(valid_df,feat_col,model,fname)

        # prepare target and pred
        target = valid_df[target_col]
        target.index, pred.index=valid_df.kinase,valid_df.kinase
        pred.columns = target.columns

        # get overall score and average score of each kinase
        score_all(target,pred)
        cor = score_each(target,pred)

        # get OOF
        OOF[split[1]] = pred.values

    OOF = pd.DataFrame(OOF,columns=target_col)
    
    print('---------- score of OOF ----------------')
    cor = score_each(y,OOF)
    return OOF, cor

In [ ]:
# def create_mlp():
#     return MLP_1(num_feat, num_target)

# mlp_params = {'n_epoch': 14, 
#           'lr_max': 3e-3,
#           'cbs': [SaveModelCallback(fname = 'best')] # save best model
#          }

# OOF, cor = train_dl_cv(df,feat_col,target_col, splits, create_mlp, params = mlp_params)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()